In [ ]:
!pip install transformers datasets accelerate


In [ ]:
import torch

# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cuda


In [ ]:
pip install pyvips

In [ ]:
!apt-get update
!apt-get install -y libvips-dev


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 129 kB in 2s (84.4 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Don

In [ ]:
from datasets import load_dataset
dataset = load_dataset("iconqa", "mc")



In [ ]:

print("Loaded dataset:", dataset)
print("Sample:", dataset["val"][0])

Loaded dataset: DatasetDict({
    val: Dataset({
        features: ['question_id', 'question', 'choices', 'answer', 'query_image', 'choice_image_0', 'choice_image_1', 'ques_type', 'label', 'grade', 'skills'],
        num_rows: 21488
    })
    test: Dataset({
        features: ['question_id', 'question', 'choices', 'answer', 'query_image', 'choice_image_0', 'choice_image_1', 'ques_type', 'label', 'grade', 'skills'],
        num_rows: 21489
    })
})
Sample: {'question_id': '1', 'question': 'Use dice to measure the line. The line is about (_) dice long.', 'choices': None, 'answer': '3', 'query_image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=466x200 at 0x7C6524439410>, 'choice_image_0': None, 'choice_image_1': None, 'ques_type': 'fill_in_blank', 'label': 'T7', 'grade': 'kindergarten', 'skills': 'measurement,estimation'}


In [ ]:
# Shuffle and select  examples from 'val' split
small_dataset = dataset["val"].shuffle(seed=42).select(range(3000))

print("Randomized small dataset example:")
print(small_dataset[0])


Randomized small dataset example:
{'question_id': '67966', 'question': 'What shape comes next?', 'choices': 'choice_0.png,choice_1.png', 'answer': 'choice_1.png', 'query_image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=533x73 at 0x7C65B1DCB710>, 'choice_image_0': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=74x74 at 0x7C64F8DB9310>, 'choice_image_1': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=74x74 at 0x7C652443B750>, 'ques_type': 'choose_img', 'label': 'H4', 'grade': 'kindergarten', 'skills': 'pattern'}


In [ ]:
# Advanced Chain-of-Thought reasoning (Better Version)
def add_advanced_cot(example):
    question = example['question']
    answer = example['answer']

    # Properly formatted detailed step-by-step reasoning
    example["input_text"] = f"""<|user|>\n{question}\nLet's work through this carefully step-by-step:
(1) First, observe all key objects and their relationships in the scene.
(2) Next, focus on the elements most relevant to answering the question.
(3) Then, reason logically about these elements based on their attributes, positions, and interactions.
(4) Finally, deduce the most accurate answer based on this logical analysis.
<|assistant|>\n{answer}"""

    return example

# Apply Chain-of-Thought formatting
small_dataset = small_dataset.map(add_advanced_cot)

print("✅ Sample after adding Advanced CoT:")
print(small_dataset[0]["input_text"])



Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

✅ Sample after adding Advanced CoT:
<|user|>
What shape comes next?
Let's work through this carefully step-by-step:
(1) First, observe all key objects and their relationships in the scene.
(2) Next, focus on the elements most relevant to answering the question.
(3) Then, reason logically about these elements based on their attributes, positions, and interactions.
(4) Finally, deduce the most accurate answer based on this logical analysis.
<|assistant|>
choice_1.png


In [ ]:
!git clone https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0



Cloning into 'TinyLlama-1.1B-Chat-v1.0'...
remote: Enumerating objects: 60, done.
remote: Total 60 (delta 0), reused 0 (delta 0), pack-reused 60 (from 1)
Unpacking objects: 100% (60/60), 519.90 KiB | 7.32 MiB/s, done.


In [ ]:
!pip install bitsandbytes==0.41.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.3 MB/s eta 0:00:00


In [ ]:
!pip install transformers --upgrade


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "gpt2-medium"

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

print("✅ GPT-2 Medium loaded successfully!")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ GPT-2 Medium loaded successfully!


In [ ]:
small_dataset = small_dataset.map(add_advanced_cot)


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
dummy_data=small_dataset


In [ ]:

from torch.utils.data import DataLoader

def collate_fn(batch):
    input_texts = [example["input_text"] for example in batch]

    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)
    labels = inputs["input_ids"].clone()

    inputs = {k: v.to(device) for k, v in inputs.items()}
    labels = labels.to(device)
    labels[labels == tokenizer.pad_token_id] = -100  # Mask padding tokens

    return inputs, labels

train_loader = DataLoader(dummy_data, batch_size=2, shuffle=True, collate_fn=collate_fn)



# train_loader = DataLoader(
#     small_dataset,   # <-- your dataset after adding CoT prompts
#     batch_size=10,
#     shuffle=True,
#     collate_fn=collate_fn
# )
from torch import autocast
from torch.cuda.amp import GradScaler
from tqdm.auto import tqdm

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scaler = GradScaler()

model.train()
num_epochs = 3

for epoch in range(num_epochs):
    total_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        optimizer.zero_grad()

        with autocast('cuda'):
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss

        if torch.isnan(loss):
            print("⚠️ Loss is NaN at this batch, skipping")
            continue

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} complete! Avg Loss = {avg_loss:.4f}")


<ipython-input-55-b84a5470cd7e>:30: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Epoch 1/3:   0%|          | 0/1500 [00:00<?, ?it/s]

✅ Epoch 1 complete! Avg Loss = 0.1570


Epoch 2/3:   0%|          | 0/1500 [00:00<?, ?it/s]

✅ Epoch 2 complete! Avg Loss = 0.0912


Epoch 3/3:   0%|          | 0/1500 [00:00<?, ?it/s]

✅ Epoch 3 complete! Avg Loss = 0.0829


In [ ]:
```
<ipython-input-55-b84a5470cd7e>:30: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/3: 100%
 1500/1500 [05:56<00:00,  4.14it/s]
✅ Epoch 1 complete! Avg Loss = 0.1570
Epoch 2/3: 100%
 1500/1500 [05:52<00:00,  4.16it/s]
✅ Epoch 2 complete! Avg Loss = 0.0912
Epoch 3/3: 100%
 1500/1500 [05:51<00:00,  4.01it/s]
✅ Epoch 3 complete! Avg Loss = 0.0829
```

In [ ]:
!pip install rouge-score


In [ ]:
!pip install nltk

In [ ]:
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

smoothie = SmoothingFunction().method4
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

total_em_direct, total_em_cot = 0, 0
total_bleu_direct, total_bleu_cot = 0.0, 0.0
total_rouge1_direct, total_rouge1_cot = 0.0, 0.0
total_rougeL_direct, total_rougeL_cot = 0.0, 0.0

model.eval()

for i in tqdm(range(10)):  # Evaluate only first 100 examples
    ground_truth = small_dataset[i]['answer']
    question = small_dataset[i]['question']

    # Inference without CoT
    inputs_direct = tokenizer(question, return_tensors="pt").to(device)
    output_direct = model.generate(**inputs_direct, max_new_tokens=50)
    pred_direct = tokenizer.decode(output_direct[0], skip_special_tokens=True)

    # Inference with CoT
    inputs_cot = tokenizer(question + "\nLet's think step-by-step:", return_tensors="pt").to(device)
    output_cot = model.generate(**inputs_cot, max_new_tokens=100)
    pred_cot = tokenizer.decode(output_cot[0], skip_special_tokens=True)

    # Clean
    pred_direct = pred_direct.strip().lower()
    pred_cot = pred_cot.strip().lower()
    ground_truth = ground_truth.strip().lower()

    # Exact Match
    total_em_direct += int(pred_direct == ground_truth)
    total_em_cot += int(pred_cot == ground_truth)

    # BLEU
    ref = [ground_truth.split()]
    total_bleu_direct += sentence_bleu(ref, pred_direct.split(), smoothing_function=smoothie)
    total_bleu_cot += sentence_bleu(ref, pred_cot.split(), smoothing_function=smoothie)

    # ROUGE
    scores_direct = scorer.score(pred_direct, ground_truth)
    scores_cot = scorer.score(pred_cot, ground_truth)

    total_rouge1_direct += scores_direct['rouge1'].fmeasure
    total_rouge1_cot += scores_cot['rouge1'].fmeasure
    total_rougeL_direct += scores_direct['rougeL'].fmeasure
    total_rougeL_cot += scores_cot['rougeL'].fmeasure

# Final scores
print("\n===== FINAL EVALUATION =====")
print(f"Avg Exact Match (Direct): {total_em_direct/100:.6f}")
print(f"Avg Exact Match (CoT): {total_em_cot/100:.6f}")
print(f"Avg BLEU (Direct): {total_bleu_direct/100:.6f}")
print(f"Avg BLEU (CoT): {total_bleu_cot/100:.6f}")
print(f"Avg ROUGE-1 (Direct): {total_rouge1_direct/100:.6f}")
print(f"Avg ROUGE-1 (CoT): {total_rouge1_cot/100:.6f}")
print(f"Avg ROUGE-L (Direct): {total_rougeL_direct/100:.6f}")
print(f"Avg ROUGE-L (CoT): {total_rougeL_cot/100:.6f}")


  0%|          | 0/10 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene


===== FINAL EVALUATION =====
Avg Exact Match (Direct): 0.000000
Avg Exact Match (CoT): 0.000000
Avg BLEU (Direct): 0.000078
Avg BLEU (CoT): 0.000426
Avg ROUGE-1 (Direct): 0.002348
Avg ROUGE-1 (CoT): 0.006029
Avg ROUGE-L (Direct): 0.002348
Avg ROUGE-L (CoT): 0.006029
